# 章節 12：證據

## 12.3 先驗分佈

大學理事會也公布所以測試者的量尺分數分佈。如果我們將每個量尺分數轉換成原始分數並且除以考試題目的數量，這個結果是估計 p_correct。所以我們能用原始分數的分佈來估計 p_correct 的先驗分佈。

以下是讀資料且處理資料的程式碼：

<pre>
class Exam(object):
    def __init__(self):
        self.scale = ReadScale()
        scores = ReadRanks()
        score_pmf = thinkbayes.MakePmfFromDict(dict(scores))
        self.raw = self.ReverseScale(score_pmf)
        self.max_score = max(self.raw.Values())
        self.prior = DivideValues(self.raw, self.max_score)
</pre>   
        
Exam 封裝關於考試的資訊。ReadScale 和 ReadRanks 讀取當案並且回傳含有資料的物件：
- self.scale 是 Interpolator 物件，可以將原始分數與量尺分數互相轉換
- scores 是一序列的配對 (score, frequency)
- score_pmf 是量尺分數的 Pmf
- self.raw 是原始分數的 Pmf
- self.prior 是 p_correct 的 Pmf

In [1]:
# TODO

上圖顯示 p_correct 的先驗分佈。這個分佈近似於高斯，但在極端值被壓縮。根據設計，SAT 對測試者具有最大的區分能力在兩個標準偏差內，而在兩個標準差範圍以外則較少的區分能力。

對每個測試者，這邊定義姥一個 Suite 子類別 Sat，用來代表 p_correct 的分佈。以下是程式碼：

<pre>
class Sat(thinkbayes.Suite):
    def __init__(self, exam, score):
        thinkbayes.Suite.__init__(self)
        self.exam = exam
        self.score = score
        
        # 從先驗分佈開始
        for p_correct, prob in exam.prior.Items():
            self.Set(p_correct, prob)
        
        # 根據考試分數更新分佈
        self.Update(score)
</pre>

\_\_init\_\_ 需要 Exam 物件跟量尺分數。它複製先驗分佈並且依據量尺分數更新分佈。

跟之前一樣，我們繼承 Suite 的 Update 方法並且提供 Likelihood 方法：
<pre>
    def Likelihood(self, data, hypo):
        p_correct = hypo
        score = data
        k = self.exam.Reverse(score)
        n = self.exam.max_score
        like = thinkbayes.EvalBinomialPmf(k, n, p_correct)
        return like
</pre>

hypo 是一個 p_correct 的假設，data 是量尺分數。

為了簡單化，這邊的原始分數當作答對的題數，忽略了答錯題目的扣分。在這種簡化的情況下，似然性是用二項次分佈來計算。它計算 n 個問題中答對 k 個問題的機率。